In [1]:
# capture --no-stderr
%pip install -U langgraph langsmith

# Used for this tutorial; not a requirement for LangGraph
%pip install -U langchain-openai
%pip install -U tavily-python
%pip install -U langchain_community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 2.3 MB 231 kB/s            
     |████████████████████████████████| 990 kB 79 kB/s             
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.1
    Uninstalling langchain-text-splitters-0.0.1:
      Successfully uninstalled langchain-text-splitters-0.0.1
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.13
    Uninstalling langchain-0.1.13:
      Successfully uninstalled langchain-0.1.13
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.29
    Uninstalling langchain-community-0.0.29:
      Successfully uninstalled langchain-community-0.0.29
Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("API_BASE_URL")
_set_env("TAVILY_API_KEY")

OPENAI_API_KEY: ········
API_BASE_URL: ········
TAVILY_API_KEY: ········


In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool.invoke("今天西安的天气如何？")

[{'url': 'http://www.weather.com.cn/weather/101110101.shtml',
  'content': '涂擦SPF大于15、PA+防晒护肤品。\n无明显降温，感冒机率较低。\n气温较低，在户外运动请注意增减衣物。\n无需担心过敏，可放心外出，享受生活。\n建议着厚外套加毛衣等服装。\n无雨且风力较小，易保持清洁度。\n辐射较弱，涂擦SPF12-15、PA+护肤品。\n大幅度降温，适当增加衣服。\n有降水，推荐您在室内进行休闲运动。\n无需担心过敏，可放心外出，享受生活。\n建议着棉衣加羊毛衫等冬季服装。\n有雨，雨水和泥水会弄脏爱车。\n辐射弱，涂擦SPF8-12防晒护肤品。\n天气资讯\n周边地区\n|\n周边景点\n2024-01-09 11:30更新\n/\n/\n/\n/\n/\n/\n/\n/\n/\n/\n/\n周边地区\n|\n周边景点\n2024-01-09 11:30更新\n/\n/\n/\n/\n/\n/\n/\n/\n/\n/\n/\n/\n高清图集\n重大天气事件\n1月9日\n冷空气难改偏暖格局 南方仍有阴雨来扰\n本周将有三股冷空气轮番登场，多地气温仍有波动，不过冷空气实力较弱且影响区域偏北，难改气温回升的大趋势。\n1月8日\n本周冷空气活跃 南方仍将多阴雨天气\n今明两天，我国降水主要出现在江南、华南以及贵州、新疆北部等地。此外，本周影响我国的冷空气仍频繁。\n1月7日\n冷空气活跃中东部多地气温频繁波动 南方多阴雨天气来扰\n今天（1月7日）至下周初，我国冷空气活动仍频繁，东北、长江中下游等地气温频繁升降，公众需留意气温变化，及时调整穿着，远离感冒风险。\n1月6日\n东北气温起伏明显 南方多地雨水频繁打卡\n今天（1月6日），受新一股冷空气影响，黄淮南部、江淮北部等地的霾天气逐渐减弱消散。同时，受其影响，东北地区将出现明显降温，后天起又将开启升温。\n1月5日\n华北江淮等地今天仍有雾和霾 东北多地迎明显降温\n今天（1月5日)，华北、黄淮、江淮等地扩散条件仍较差，或现轻至中度霾，明天起冷空气上线，霾天气将逐渐消散。东北多地受冷空气影响还将出现明显降温。\n1月4日\n冷空气实力较弱全国大部气温偏高 东北内蒙古迎降雪\n尽管气温起起伏伏，但总体来看，未来一周全国大部气温仍然以偏暖为主

In [6]:

from typing import Annotated

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


api_key = os.environ.get("OPENAI_API_KEY")
base_url = os.environ.get("API_BASE_URL")

llm = ChatOpenAI(model="gpt-4o-mini",openai_api_key=api_key, openai_api_base=base_url)
# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

In [7]:
import json
from langchain_core.messages import ToolMessage
class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}


tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

In [8]:
from typing import Literal


def route_tools(
    state: State,
) -> Literal["tools", "__end__"]:
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "__end__"

# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "__end__" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", "__end__": "__end__"},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [9]:

from langchain_core.messages import BaseMessage

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                print("Assistant:", value["messages"][-1].content)

User: 今天宝鸡天气如何？
Assistant: 
Assistant: "HTTPError('400 Client Error: Bad Request for url: https://api.tavily.com/search')"
Assistant: 
Assistant: [{"url": "http://www.weather.com.cn/weather/101110901.shtml", "content": "\u5b9d\u9e21\u5929\u6c14\u9884\u62a5\uff0c\u53ca\u65f6\u51c6\u786e\u53d1\u5e03\u4e2d\u592e\u6c14\u8c61\u53f0\u5929\u6c14\u4fe1\u606f\uff0c\u4fbf\u6377\u67e5\u8be2\u5b9d\u9e21\u4eca\u65e5\u5929\u6c14\uff0c\u5b9d\u9e21\u5468\u672b\u5929\u6c14\uff0c\u5b9d\u9e21\u4e00\u5468\u5929\u6c14\u9884\u62a5\uff0c\u5b9d\u9e21\u84dd\u5929\u9884\u62a5\uff0c\u5b9d\u9e21\u5929\u6c14\u9884\u62a5\uff0c\u5b9d\u9e2140\u65e5\u5929\u6c14\u9884\u62a5\uff0c\u8fd8\u63d0\u4f9b\u5b9d\u9e21\u7684\u751f\u6d3b\u6307\u6570\u3001\u5065\u5eb7\u6307\u6570\u3001\u4ea4\u901a\u6307\u6570\u3001\u65c5\u6e38\u6307\u6570\uff0c\u53ca\u65f6\u53d1\u5e03\u5b9d\u9e21\u6c14\u8c61\u9884\u8b66\u4fe1\u53f7\u3001\u5404\u7c7b\u6c14\u8c61\u8d44\u8baf\u3002"}, {"url": "http://www.nmc.cn/publish/forecast/ASN/baoji.html", "cont

In [10]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
api_key = os.environ.get("OPENAI_API_KEY")
base_url = os.environ.get("API_BASE_URL")

llm = ChatOpenAI(model="gpt-4o-mini",openai_api_key=api_key, openai_api_base=base_url)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [11]:
from langchain_core.messages import BaseMessage

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                print("Assistant:", value["messages"][-1].content)

User: Hi
Assistant: Hello! How can I assist you today?
User: 今天Yum公司美股收盘价是多少
Assistant: 
Assistant: [{'url': 'https://finance.yahoo.com/quote/YUM/history/', 'content': 'Discover historical prices for YUM stock on Yahoo Finance. View daily, weekly or monthly format back to when Yum! ... Close price adjusted for splits. ... Oct 18, 2023: 119.47: 121.56: 119.35: 120 ...'}, {'url': 'https://www.nasdaq.com/market-activity/stocks/yum/historical', 'content': 'Back to YUM Overview. Get up to 10 years of daily historical stock prices & volumes. The "Close/Last" is the "adjust consolidated close price". Data provided by Edgar Online . The net and ...'}]


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'messages[2].content[0].type'. (request id: 2024072214453645900806194353942)", 'type': 'invalid_request_error', 'param': 'messages[2].content[0].type', 'code': 'missing_required_parameter'}}

# 给 chatBot 赋予记忆

In [12]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [13]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
api_key = os.environ.get("OPENAI_API_KEY")
base_url = os.environ.get("API_BASE_URL")

llm = ChatOpenAI(model="gpt-4o-mini",openai_api_key=api_key, openai_api_base=base_url)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [15]:
config = {"configurable": {"thread_id": "1"}}

In [16]:
user_input = "看这里，你好我是小刘同学."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

看这里，你好我是小刘同学.
================================== Ai Message ==================================

你好，小刘同学！有什么我可以帮助你的吗？


In [17]:

user_input = "你还记得我叫什么吗?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

你还记得我叫什么吗?
================================== Ai Message ==================================

当然记得，你叫小刘同学！有什么想和我聊的呢？


In [18]:

# The only difference is we change the `thread_id` here to "2" instead of "1"
events = graph.stream(
    {"messages": [("user", user_input)]},
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

你还记得我叫什么吗?
================================== Ai Message ==================================

抱歉，我无法记住或存储个人信息，包括你的名字。如果你告诉我你的名字，我可以在这个对话中使用它。


In [19]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='看这里，你好我是小刘同学.', id='8d8fbd41-1ed5-4780-938b-f494770e4b2a'), AIMessage(content='你好，小刘同学！有什么我可以帮助你的吗？', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 89, 'total_tokens': 103}, 'model_name': 'openai/gpt-4o-mini', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-36a137aa-d3d7-4613-9ac0-e8b5de5ba37e-0', usage_metadata={'input_tokens': 89, 'output_tokens': 14, 'total_tokens': 103}), HumanMessage(content='你还记得我叫什么吗?', id='f98b95f7-0696-41bf-9160-07094de89ef2'), AIMessage(content='当然记得，你叫小刘同学！有什么想和我聊的呢？', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 118, 'total_tokens': 137}, 'model_name': 'openai/gpt-4o-mini', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-b4bdff38-fab2-46e8-b37f-b4a587cec665-0', usage_metadata={'input_tokens': 118, 'output_tokens': 19, 'total_tokens': 137})]}, next=(), con